The data contains a pair of paragraphs. These text paragraphs are randomly sampled from a raw dataset. Each pair of the sentence may or may not be semantically similar. The dataset considered for this project does not contain any labels. Given below is the solution to the unsupervised machine learning problem

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import re
import scipy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec, KeyedVectors, FastText
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Flatten, SimpleRNN, RNN,GRU, SpatialDropout1D, Dropout
from keras.preprocessing.sequence import pad_sequences
import sklearn


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data=pd.DataFrame()
data = pd.read_csv('../input/open-iit-dataset/train.csv')
data.shape #(4023,3)
#data.head()
#print(data.shape)
#print(data.head())
dictionary={}
for word in range(data.shape[0]):
  dictionary[data['Id'][word]]=data['Text'][word]

# print(dictionary)
def access_text_using_id(id_no):
  print(dictionary[id_no])

# access_text_using_id(57)


Train=pd.read_csv('../input/open-iit-dataset/train_data.csv')
X = Train.iloc[:,0:].values
s1 = X[:,0]
s2 = X[:,1]
s5 = X[:,2]

s3=[]
s4=[]
for s in s1:
    s3.append(dictionary[s])
index=0
for s in s2:
    s4.append(dictionary[s])
# for i in range (5):
#     print(s3)

In [3]:
print(s5)

[0 1 0 ... 0 1 0]


In [4]:
print(s3[3])
print(s4[3])

She will not be able to understand all these aspects in a day .
He must possess all the qualifications necessary to become a Member of Lok Sabha .


In [5]:
# df = pd.read_csv('../input/comparisonwords20/ComparisonWords_big (1).csv') 
# mylist_compare = df['Words'].tolist()
# print(mylist_compare)

df = pd.read_csv('../input/masculine-feminine-list/Masculine.csv')
mylist_masculine = df['Male'].tolist()

df=pd.read_csv('../input/masculine-feminine-list/Feminine.csv')
mylist_feminine = df['Female'].tolist()
# print(mylist_gender)

# def check_comparison_word(word):
#     if word in mylist_compare:
#         return 1
    
def check_masculine_word(word):
    if word in mylist_masculine:
        return 1
    
def check_feminine_word(word):
    if word in mylist_feminine:
        return 1
    

# Preprocessing of Data

In [6]:
tokens1 = []
tokens2 = []
tokens1 = [word_tokenize(str(sentence)) for sentence in s3]
tokens2 = [word_tokenize(str(sentence)) for sentence in s4]

rm1 = []
for w in tokens1:
    sm = re.sub('[^A-Za-z]',' ', str(w))
    x = re.split("\s", sm)
    rm1.append(x)
    
rm2 = []
for w in tokens2:
    sm2 = re.sub('[^A-Za-z]',' ', str(w))
    x2 = re.split("\s", sm2)
    rm2.append(x2)


In [29]:
print(rm1[3])
print(rm2[3])

['She', 'will', 'not', 'be', 'able', 'to', 'understand', 'all', 'these', 'aspects', 'in', 'a', 'day']
['He', 'must', 'possess', 'all', 'the', 'qualifications', 'necessary', 'to', 'become', 'a', 'Member', 'of', 'Lok', 'Sabha']


In [7]:
#Removing whitespaces    
for sent in rm1:
    while '' in sent:
        sent.remove('')

for sent in rm2:
    while '' in sent:
        sent.remove('')
        
# Lowercasing
low1 = []
for i in rm1:
    i = [x.lower() for x in i]
    low1.append(i)

low2 = []
for i in rm2:
    i = [x.lower() for x in i]
    low2.append(i)


In [21]:
print(low1[3])
print(low2[3])

['she', 'will', 'not', 'be', 'able', 'to', 'understand', 'all', 'these', 'aspects', 'in', 'a', 'day']
['he', 'must', 'possess', 'all', 'the', 'qualifications', 'necessary', 'to', 'become', 'a', 'member', 'of', 'lok', 'sabha']


In [8]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [9]:
#Lemmatozation
lemma1 = []
wnl = WordNetLemmatizer()
for sent in low1:
    tokens = [wnl.lemmatize(w) for w in sent]
    lemma1.append(tokens)
    
lemma2 = []
for sent in low2:
    tok = [wnl.lemmatize(se) for se in sent]
    lemma2.append(tok)
    
# Removing Stopwords
filter_words1 = []
Stopwords = set(stopwords.words('english'))

for sent in lemma1:
    tokens = [w for w in sent if w not in Stopwords or w in mylist_masculine or w in mylist_feminine]
    filter_words1.append(tokens)
    
filter_words2 = []
for sent in lemma2:
    tokens2 = [w for w in sent if w not in Stopwords or w in mylist_masculine or w in mylist_feminine]
    filter_words2.append(tokens2)


In [40]:
print(filter_words2[22])
print(filter_words1[22])

['collect', 'feather', 'different', 'bird', 'decorate', 'peacock', 'pasting', 'feather']
['flow', 'chart', 'showing', 'event', 'sexual', 'reproduction', 'male', 'parent', 'female', 'parent', 'gamete', 'production', 'male', 'gamete', 'female', 'gamete', 'fusion', 'fertilization', 'zygote', 'embryo', 'young', 'individual', 'organism', 'reproduce', 'sexual', 'method']


In [13]:
count_of_ones_male=0
count_of_ones_female=0
def check_para_for_male(para):
    global count_of_ones_male
    for word in para:
        count_of_ones_male+=1
        if(check_masculine_word(word)==1):
#             count_of_ones+=1
            return 1
    return 0

def check_para_for_female(para):
    global count_of_ones_female
    for word in para:
        count_of_ones_female+=1
        if(check_feminine_word(word)==1):
#             count_of_ones+=1
            return 1
    return 0

gender_check1=[]
for text in filter_words1:
    count_of_ones_male=0
    count_of_ones_female=0
    if(check_para_for_male(text) == check_para_for_female(text)):
#         gender_check1.append(0)
        if(check_para_for_male(text) == 1):
            if(count_of_ones_male < count_of_ones_female):
                gender_check1.append(1)
            else:
                gender_check1.append(0)
        else:
            gender_check1.append(0)
            
    else:
        gender_check1.append(1)
    """
    masculine + feminine = unbiased(0)
    only masculine or only feminine = biased(1)
    """
    
#print("Set1 Gender count: Number of ones:", count_of_ones, "Number of zeroes: ", len(gender_check1)-count_of_ones)
    
count_of_ones=0
gender_check2=[]
for text in filter_words2:
#     gender_check2.append(int(not(check_para_for_male(text) and check_para_for_female(text))))
    if(check_para_for_male(text) == check_para_for_female(text)):
#         gender_check2.append(0)
        if(check_para_for_male(text) == 1):
            if(count_of_ones_male < count_of_ones_female):
                gender_check2.append(1)
            else:
                gender_check2.append(0)
        else:
            gender_check2.append(0)   
    else:
        gender_check2.append(1)
    
#print("Set2 Gender Count: Number of ones:", count_of_ones, "Number of zeroes: ", len(gender_check1)-count_of_ones)
    
length_of_dataset=len(gender_check1)
# print(gender_check2)
# print(gender_check1)



In [15]:
# print(comparison_check2[73])
print(filter_words2[4])

['he', 'spends', 'his', 'earned', 'money', 'buy', 'essential', 'good', 'satisfy', 'his', 'want', 'consumption', 'good']


Set1 Gender and Comparison check: Number of ones 39555 Number of zeroes 116396
Set2 Gender and Comparison check: Number of ones 14055 Number of zeroes 141896


In [16]:
count_of_ones=0
gender_and_compare_final=[]
for i in range(length_of_dataset):
    if(gender_check1[i] == gender_check2[i]):
        gender_and_compare_final.append(0)
    else:
        gender_and_compare_final.append(1)
        count_of_ones+=1
print("Number of ones", count_of_ones, "Number of zeroes", length_of_dataset-count_of_ones)
# print(gender_and_compare_final)

Number of ones 76542 Number of zeroes 79409


# FastText
FastText is an extension to Word2Vec proposed by Facebook in 2016. Instead of feeding individual words into the Neural Network, FastText breaks words into several n-grams (sub-words). For instance, the tri-grams for the word apple is app, ppl, and ple (ignoring the starting and ending of boundaries of words). The word embedding vector for apple will be the sum of all these n-grams. After training the Neural Network, we will have word embeddings for all the n-grams given the training dataset. Rare words can now be properly represented since it is highly likely that some of their n-grams also appears in other words.


## Cosine Similarity
Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. The cosine of 0° is 1, and it is less than 1 for any angle in the interval (0, π] radians.


In [17]:
#making a new row
Train['Firstness'] = gender_and_compare_final

In [18]:
#Getting an output file
Train.to_csv("output.csv")

In [19]:
#Accuracy
sklearn.metrics.accuracy_score(s5, gender_and_compare_final)


0.8412770677969362